# Association Rules

- Association Rules: $X -> Y$, when item X is observed, item Y is also observed

- Association Rules are sometines referred to as *market bascket analysis*.

- A collection of items, like {'milk', 'bread'}: *itemset*

- An itemset containing *k* items: *k-itemset*

## Support, Confidence and Lift

### Support

- The proportion of transactions that contain both A and B.

- In other words, how often a itemset, such as {'milk', 'bread'}, appears in a transaction (or a trial).

### Confidence

- The probability that B is purchased given A is purchased.

### Lift

- The ratio of the observed support to the expected support if A and B were independent.

- $Support/Confidence$

### Probability theory

- Association rules in data mining are related to basic probability theory, and especially to contingency tables

- It is easier to understand the idea of the association theory if you know the contingency table.

- Association rules aim to find patterns like:

  > "If item A is bought, then item B is likely to be bought."
(e.g., If a customer buys bread, they are likely to buy butter.)

### Probability theory and association rule

- This kind of statement is inherently probabilistic and is quantified using three key metrics:

- **Support**:Probability that both A and B occur (joint probability)

$$
\large{P(A \cap B)}
$$

- **Confidence**: Probability of B given A (conditional probability)

$$
\large{P(B \mid A) = \frac{P(A \cap B)}{P(A)}}
$$

- **Lift**: How much A increases the chance of B (not mutually exclusive (Dependent check))

$$
\large
\frac{P(B \mid A)}{P(B)} = \frac{P(B \cap A)}{P(A)P(B)} = Confidence/Support (B)
$$

### Interpretation of Lift

- Lift = 1 → A and B are independent.

- Lift > 1 → A and B are positively correlated (B is more likely to occur with A).

- Lift < 1 → A and B are negatively correlated.
These are all grounded in basic probability concepts.



## Basic Algorithm: Apriori and FR-Growth


### Apriori

- Apriori Algorithm:  Apriori is a breadth-first, candidate generation algorithm that uses the Apriori property:
If an itemset is frequent, then all its subsets must also be frequent.

🧠 How it works (step-by-step):

- Count item frequencies and find frequent 1-itemsets (those that meet a minimum support threshold).

- Generate candidate 2-itemsets from frequent 1-itemsets.

- Scan the database to count their support and prune non-frequent ones.

- Repeat for k-itemsets until no more frequent sets are found.

✅ Advantages: Simple and intuitive.

- Easy to implement.

❌ Disadvantages:

- Computationally expensive, especially with large datasets.

- Requires multiple scans of the database.

- Generates many candidate sets, including useless ones.

### FR-Growth

- FP-Growth (Frequent Pattern Growth) FP-Growth is a divide-and-conquer algorithm that improves efficiency by avoiding candidate generation altogether.


🧠 How it works:

- Build a compact data structure called the FP-tree (Frequent Pattern tree), a compressed representation of transactions.

- Traverse the tree recursively to extract frequent itemsets using pattern growth.

✅ Advantages:

- Much faster than Apriori, especially for dense or large datasets.

- Requires only two scans of the database.

- Avoids candidate generation.

❌ Disadvantages:

- FP-tree construction can be complex to implement.

- May use more memory if the tree is large.

### Comparison

| Feature           | Apriori                        | FP-Growth                |
| ----------------- | ------------------------------ | ------------------------ |
| Strategy          | Candidate generation + pruning | Pattern growth           |
| DB scans          | Multiple                       | Only two                 |
| Memory efficiency | Lower                          | Higher (but faster)      |
| Suitable for      | Small to medium datasets       | Large and dense datasets |
| Output            | Frequent itemsets              | Frequent itemsets        |

### Summary

| Algorithm     | Key Idea                                   | Fast? | Memory?  |
| ------------- | ------------------------------------------ | ----- | -------- |
| **Apriori**   | Generates and tests candidate itemsets     | ❌     | ✅ Low    |
| **FP-Growth** | Builds tree and mines patterns recursively | ✅     | ❌ Higher |


In [ ]:
# First install the mlxtend package
!pip install mlxtend

In [ ]:
# Import necessary libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

## Grocery dataset

### Creating simple dataset

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import random

# Define the base correlated items and extra noise items
itemset = ['milk', 'bread', 'apple']
extra_items = ['eggs', 'butter', 'nuts', 'juice', 'cheese', 'chocolate', 'yogurt', 'banana', 'chicken']

# Create synthetic transactions
random.seed(42)
trials = []
for _ in range(100):
    basket = itemset.copy() if random.random() < 0.70 else []  # 70% support
    basket += random.sample(extra_items, random.randint(1, 3))
    trials.append(basket)

# Encode transactions
te = TransactionEncoder()
te_ary = te.fit(trials).transform(trials)
df = pd.DataFrame(te_ary, columns=te.columns_)

print(df)


    apple  banana  bread  butter  cheese  chicken  chocolate   eggs  juice  \
0    True   False   True   False    True    False      False  False  False   
1    True   False   True    True   False    False      False  False  False   
2    True   False   True    True   False    False      False   True  False   
3    True   False   True   False   False    False      False  False   True   
4    True   False   True   False   False     True      False  False  False   
..    ...     ...    ...     ...     ...      ...        ...    ...    ...   
95   True   False   True    True    True    False      False  False   True   
96  False   False  False   False   False    False      False  False  False   
97  False   False  False   False   False     True      False  False   True   
98  False   False  False   False    True    False       True  False  False   
99  False   False  False   False   False    False      False  False   True   

     milk   nuts  yogurt  
0    True  False   False  
1    True

### Using Apriori to Find Patterns

In [ ]:
# Step 1: Find frequent itemsets with at least 60% support
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.71,(apple)
1,0.71,(bread)
2,0.71,(milk)
3,0.71,"(bread, apple)"
4,0.71,"(milk, apple)"
5,0.71,"(bread, milk)"
6,0.71,"(bread, milk, apple)"


In [ ]:
# Step 2: Generate rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

,antecedents,consequents,support,confidence,lift
0,(bread),(apple),0.71,1.0,1.408451
1,(apple),(bread),0.71,1.0,1.408451
2,(milk),(apple),0.71,1.0,1.408451
3,(apple),(milk),0.71,1.0,1.408451
4,(bread),(milk),0.71,1.0,1.408451
5,(milk),(bread),0.71,1.0,1.408451
6,"(bread, milk)",(apple),0.71,1.0,1.408451
7,"(bread, apple)",(milk),0.71,1.0,1.408451
8,"(milk, apple)",(bread),0.71,1.0,1.408451
9,(bread),"(milk, apple)",0.71,1.0,1.408451


### FP-Growth Algorithm

In [ ]:
# Same process using FP-Growth instead of Apriori
fp_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
rules_fp = association_rules(fp_itemsets, metric="confidence", min_threshold=0.7)
rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

### Downward Closure Property or the Apriori Property

The **Downward Closure Property** (also called the Apriori Property) states:
> If an itemset is frequent, then all of its subsets must also be frequent.

**Example:**
If `{'milk', 'bread', 'apple'}` appears in 60% of transactions, then:
- `{'milk', 'bread'}` must also appear in at least 60%
- `{'milk', 'apple'}`, `{'bread', 'apple'}`, `{'milk'}`, etc., must also be frequent

This property helps Apriori to **prune** itemsets early, reducing the number of combinations to check.


### Redundant Rules
Some rules may be **redundant** — they don’t give new information.

**Example:**
Both of these rules may appear:
- `{'bread'} → {'milk'}`
- `{'bread', 'apple'} → {'milk'}`

The second rule might not add new insight if the first one is already strong.  
You can remove rules that have the same consequent and a subset of antecedents with equal or lower confidence.
